In [1]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd
import os

import yaml
import hashlib

In [2]:
output_dir = '/Users/kumiori3/Documents/WIP/Nature/paper_IJTCAM/technology/fig'

In [16]:
class SymbolicDiscreteDamage:
    def __init__(self, matpar, N = 1, 
                 name = "discrete generalised damage",
                 slug = "pw-linear"):
        self.matpar = matpar
        self.state = self.get_state(N)
        self.str = name
        self.slug = slug

    def get_state(self, N):
        u = sp.symbols([f"u{i}" for i in range(0, N)])
        e = sp.symbols([f"e{i}" for i in range(1, N+1)])
        α = sp.symbols([f"α{i}" for i in range(1, N+1)])

        state = {"u": u, "α": α, "e": e}
        return state

    def _s(self, α):
        s = sp.Function('s')
        return s(α)

    def _a(self, α):
        a = sp.Function('a')
        return a(α)
    
    def _w(self, αi):
        w = sp.Function('w')

        return w(αi)

    def energy(self):
        """Total energy"""
        return self._elastic() + self._damage() - self.work()

    def work(self):
        return 0

    def _elastic(self):
        state = self.state
        e = state["e"]
        α = state["α"]

        _E0 = self.matpar["E0"]
        _N = self.matpar["N"]
    
        return 1/_N * sum([1./2. * _E0 * self._a(α[i]) * e[i]**2 for i in range(len(α))])

    def _damage(self):
        state = self.state
        _α = state["α"]
        _w1 = self.matpar["w1"]

        _N = self.matpar["N"]

        return 1/_N * sum([_w1 * self._w(_α[i]) for i in range(len(_α))])

    # Explicit expressions

    def a(self, α):
        γ = self.matpar["γ"]
        return (1 - self._w(α)) / ((γ-1) * self._w(α) + 1)

    def w(self, α):
        return α

    def s(self, α):
        return 1. / self.a(α)

    def __str__(self):
     return self.str

In [14]:
class ModelAnalysis:
    """Helper class to analyse (stability) properties of a model"""

    def __str__(self):
        return self.model.slug

    def __init__(self, model):
        self.model = model
        self.state = model.state
        self.matpar = model.matpar

        _β = sp.symbols("β")
        _e = sp.symbols("e")
        self._β = _β
        self._e = _e

        self._s = model._s(_β)
        self._sp = sp.diff(self._s, _β, 1)
        self._spp = sp.diff(self._s, _β, 2)

        self._a = model._a(_β)
        self._ap = sp.diff(self._a, _β, 1)
        self._app = sp.diff(self._a, _β, 2)

        self._w = model._w(_β)
        self._wp = sp.diff(self._w, _β)
        self._wpp = sp.diff(self._w, _β, 2)

        # self.criterion()


In [15]:
N = sp.symbols("N")
t = sp.symbols('t')
γ = sp.symbols('γ')
E0, L, w1, σc = sp.symbols('E0 L w1 σc')

_N = 2
_normalise = {E0: 1, w1: 1, L: 1}

_α = sp.symbols("α")
β = sp.symbols('β')


_matpar = {N: _N, γ: 5, E0: 1, w1: 1, L: 1}
matpar = {"N": N, "γ": γ, "E0": E0, "w1": w1, "L": L}

atls = SymbolicDiscreteDamage(matpar, _N)
model = ModelAnalysis(atls)

# model.criterion()

